In [1]:
import pandas as pd
import lxml.html as lh
import requests
import time
import os
import igc_validation

In [92]:
def download_flight(flight_number):
    
    filename = "flights/flight_{}.igc".format(flight_number)
    print(filename)
    
    headers = {'User-Agent': 'Mozilla/5.0'}

    payload = {'type':'igc',
               'flightID': str(flight_number),
               'file': '',
               'captchaStr': '',
               'captcha': '',
              }

    session = requests.Session()
    res = session.post('http://xc.rmhpa.org/download_igc.php', headers=headers, data=payload)


    with open(filename, 'wb') as outfile:
        outfile.write(res.content)
     
    
    logbook = []
    logbook.append({'igcfile': os.path.abspath(filename)})
    try:
        for flight in logbook:
            flight = igc_validation.parse_igc(flight)

        for flight in logbook:
            flight = igc_validation.crunch_flight(flight)

        for flight in logbook:
            flight['outputfilename'] = igc_validation.get_output_filename(flight['igcfile'])
    except:
        filename = "flights_invalid/flight_{}.igc".format(flight_number)
        with open(filename, 'wb') as outfile:
            outfile.write(res.content)

This will take about 3 hours... I have about 200 of them.

In [93]:
df = pd.read_csv("clean/rmhpa_flights.csv")
records = df.to_dict(orient='records')
tic = time.time()

for i, rec in enumerate(records):
    url = rec['links']
    flight_number = url.split('/')[-1]
    
    if not os.path.exists("flights/flight_{}.igc".format(flight_number)): 
        print("---------------------------------------------")
        download_flight(flight_number)
        print(" #{} {} {}    {}% done....   {} seconds".format(flight_number, str(rec['index']), rec['distance'], round(i/len(records)*100), round(time.time()-tic)))
        time.sleep(1)


---------------------------------------------
flights/flight_971.igc
Flight date: 2015-04-11
Header (not implemented): OPLTPILOT: Jan Voegeli
Header (not implemented): OGTYGLIDERTYPE: R12
Header (not implemented): OGIDGLIDERID:
Header (not implemented): ODTM100GPSDATUM: WGS-84
Header (not implemented): OCIDCOMPETITIONID:
Header (not implemented): OCCLCOMPETITION CLASS:
Header (not implemented): OSITSite:
Record Type L not implemented: XGD GpsDumpMac version 0.30
Record Type L not implemented: XGD Downloaded 2015-04-12  05:00:07
Record Type G not implemented: 7DFCA5D8933B39FB
 #971 1 177.8 km    0% done....   4 seconds
---------------------------------------------
flights/flight_621.igc
Flight date: 2013-08-16
Header (not implemented): OPLTPILOT: Jan Voegeli
Header (not implemented): OGTYGLIDERTYPE: Icepeak 6
Header (not implemented): OGIDGLIDERID:
Header (not implemented): ODTM100GPSDATUM: WGS-84
Header (not implemented): OCIDCOMPETITIONID:
Header (not implemented): OCCLCOMPETITION CLA

NameError: name 'xrange' is not defined